# Projekt: Analiza zależności między przewlekłymi dolegliwościami po operacjach endometriozy a czynnikami środowiskowymi, rodzajem operacji i trybem życia.

1. **Cel projektu:** zbadanie czy są zależności między występowaniem przewlekłych dolegliwości po operacjach endometriozy nie występujących przed tymi operacjami, a czynnikami takimi jak rodzaj operacji, środowisko, czy są dodatkowe metody wspomagające organizm, trybem życia itp. oraz rodzajów dolegliwości

2. Analiza zależności między przewlekłymi dolegliwościami po operacjach endometriozy, a czynnikami środowiskowymi, rodzajem operacji i trybem życia

3. **Hipotezy:** Moja hipoteza jest taka, że:
- przewlekłe dolegliwości po operacjach są rzadkie,
- przewlekłe dolegliwości po operacjach są zależne od zakresu operacji,
- przewlekłe dolegliwości po operacjach są bardzo zróżnicowane, ale niezależne od stosowanych diet, fizjoterapii i innych sposobów na poprawianie stanu organizmu, występują niezależnie od istnienia współistniejących chorób oraz są niezależne od stanu środowiska naturalnego w miejscu zamieszkania.

4. **Metody:** W celu zebrania danych potrzebnych do analizy i budowy modelu uczenia maszynowego sporządziłam odpowiednią ankietę i udostępniłam ją na grupach na Facebooku dla osób chorych na endometriozę oraz została udostępniona na stronie Fundacji Pokonać Endometriozę. Po zebraniu odpowiedniej ilości wypełnionych ankiet, przystąpiłam do przeprowadzenia analizy danych używając języka Python. 

Kolejne kroki do wykonania:
1. Wykorzystać mały model LLM oraz spróbować wykorzystać jednostki nazwane NER z scispacy -> tam gdzie są odpowiedzi na pytania otwarte, żeby z każdego rekordu wyciągnąć najważniejsze informacje w postaci list (osoby wypełniające ankietę w bardzo różny sposób odpowiadają na pytania, zakresy operacji są różne, różne są dolegliwości).
2. Aby zmniejszyć liczbę wymiarów usunąć niepotrzebne kolumny, które po wstępnej analizie statystycznej na pewno nie będą miały wpływu na ryzyko pojawienia się dolegliwości. Połączyć dane z niektórych kolumn, aby uzyskać ważniejszą informację i zarazem też zmniejszyć ilość wymiarów, np. zamiast osobno waga i wzrost, to obliczyć BMI więc już będzie jedna kolumna, a nie dwie, itp.
4. Wykorzystać reguły asocjacyjne - czyli wnioskowanie o współwystępowaniu poszczególnych zjawisk. Jeśli występuje A To występuje B w x[%] przypadków. Algorytmy i narzędzia w bibliotece mlxtend - algorytm budowania reguł asocjacyjnych https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/. 
Zrobić analizę:
- jakie są zależności między różnymi czynnikami a występowaniem dolegliwości?
5. Na koniec zrobić interaktywny Dashboard z wykresami w PowerBI.

# Ładowanie bibliotek

In [1]:
import pandas as pd
import numpy as np

#potrzebne do kodu, który będzie tłumaczył na angielski:
from langchain_openai import ChatOpenAI				
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

# Ustawienie sposobu wyświetalania zawartości dataframe.
Ustawienie żeby wyświetlała się pełna zawartość komórek w dataframe i wszystkie kolumny (aby łatwiej mi było sprawdzać zawartość).

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# Wczytanie surowych danych

In [ ]:
df = pd.read_csv("C:/Users/justy/Documents/Projekt/Rodzaj i ryzyko wystąpienia przewlekłych dolegliwości po operacjach endometriozy.csv")  #wczytanie danych
df.head()                 #sprawdzenie pierwszych 5 wierszy co znajduje się we wczytanej tabeli

# Wstępna obróbka wczytanych danych

Nagłówki kolumn zawierają bardzo długie nazwy. Jest to związane ze sposobem sporządzenia ankiety, zakładającym, że ankieta miała być jak najbardziej zrozumiała dla osób wypełniających kwestionariusze (stąd obszerne objaśnienia jak wypełniać poszczególne pola ankiety).

## W następnym kroku jest zamiana długich opisowych nagłówków kolumn, na krótkie:

In [ ]:
# Lista nowych nazw kolumn
nowe_nazwy = ["Timestamp", 
              "Rozpoznanie_pooperacyjne", 
              "Zakres_operacji", 
              "Czy_dolegliwosci", 
              "Rodzaje_dolegliwosci", 
              "Rodzaj_operacji", 
              "Czy_leczenie_hormonalne", 
              "Czy_dietetyk", 
              "Okres_dietetyk", 
              "Czy_dieta_własna", 
              "Czy_Fizjoterapeuta_po_operacji", 
              "Okres_fizjoterapeuta", 
              "Czy_szpital_uzdrowiskowy", 
              "Aktywnosc_fizyczna", 
              "Czy_inne_metody", 
              "Jakie_inne_metody", 
              "Czy_warzywa_i_owoce_5_porcji", 
              "Czy_inne_choroby", 
              "Jakie_inne_choroby", 
              "Ile_operacji", 
              "Czy_operacja_zwiadowcza", 
              "Ile_po_operacji", 
              "Wiek", 
              "Waga", 
              "Wzrost", 
              "Miasto_wies", 
              "Czy_zanieczyszczenia_przemyslowe", 
              "Czy_smog"]

# Zmień nazwy kolumn
data = df.rename(columns=dict(zip(df.columns, nowe_nazwy)))
data.head()

Część danych w niektórych kolumnach zawiera dużo NaN, przy czym jest to związane, z tym, że respondent nie musiał wypełniać danego pola, gdyż to pole jest zależne od odpowiedzi na wcześniejsze pytanie. To znaczy, że jeśli w pytaniu X respondent odpowiedział TAK, to musiał odpowiedzieć na pytanie Y, ale jeśli w pytaniu X respondent odpowiedział NIE, wtedy nie musiał odpowiadać na pytanie Y. Wierszy z tymi wartościami nie wolno usuwać, gdyż niosą one wiedzę.

Przed przystąpieniem do obróbki danych i ich analizy w Python, w pierwszym kroku otwarłam dane w MS Excel i przejrzałam je.
Zauważyłam, że osoby wypełniające ankiety czasem nie wpisywały rozpoznania operacyjnego, a jedynie słowa typu: "To jest wyżej", "tjw.". Wynika to z tego, że najpierw musiały wpisać informacje jakie było rozpoznanie pooperacyjne i prawdopodobnie przyjęły, że to co wykonano podczas operacji równa się temu co rozpoznano podczas operacji. 

W tym podobnych przypadkach do takich komórek (tam gdzie są słowa typu "tjw." itp.) przekopiuję to co te osoby wpisały jako odpowiedź do rozpoznania pooperacyjnego. Ingerencja ta podyktowana jest tym, że tekst w stylu "To jest wyżej" nie daje wartościowych informacji dla późniejszej automatycznej analizy.

## Uzupełnienie informacji w komórkach z tekstami typu: "to jest wyżej":

In [ ]:
# Po przeglądnięciu danych w MS Excel wiem, że "tjw." jest między innymi w 7 wierszu od końca. Poniżej wyświetlam jak wygląda ostatnie 7 wierszy dataframu:
data.tail(7)

W MS Excel sprawdziłam jakie teksty są wpisywane w kolumnie "Zakres_operacji", które nie są opisem zakresu operacji (teksty typu "tjw."). Znalezione ręcznie teksty wykorzystam do funkcji do aktualizacji kolumny "Zakres_operacji" jak opisałam wyżej.

In [ ]:
# Funkcja do aktualizacji wartości w kolumnie 'Zakres_operacji'
def update_Zakres_operacji(row):
    if pd.notna(row['Zakres_operacji']) and any(phrase in row['Zakres_operacji'] for phrase in [
        'Wszystko jak wyżej',
        'Wpisałam wyżej',
        'Tjw.',
        'Laparoskopia z mini laparotomia']):
        # Dodanie wartości z kolumny Rozpoznanie_pooperacyjne do kolumny Zakres_operacji:
        additional_text = '' if pd.isna(row['Rozpoznanie_pooperacyjne']) else row['Rozpoznanie_pooperacyjne']
        return f"{row['Zakres_operacji']}:{additional_text}"
    else:
        return row['Zakres_operacji']

# Zastosowanie funkcji do aktualizacji kolumny Zakres_operacji
data['Zakres_operacji'] = data.apply(update_Zakres_operacji, axis=1)

# Sprawdzenie czy wszystko jest prawidłowo (czy do kolumny 'Zakres_operacji' jest dodany opis z kolumny 'Rozpoznanie_pooperacyjne') a wiem, że na dole dataframu były takie przypadki, że nie był wpisany zakres operacji
data.tail(7)

Po sprawdzeniu jednego z wierszy, tu o indeksie 51, widzę, że kolumna Zakres_operacji jest poprawnie zaktualizowana.

# Tłumaczenie na język angielski

W związku z tym, że język polski ma bardzo niewielkie wsparcie w modelach językowych, a dodatkowo jest użyty nie tylko język polski, dane opisowe znajdujące się w danych, najpierw zostaną przetłumaczone na język angielski.
Dzięki temu będzie można skorzystać z biblioteki scispacy do wyodrębniania jednostek nazwanych NER odnoszących się do dziedzin medycznej i biologicznej. W późniejszych krokach sprawdzę, czy jednostki nazwane NER pomogą w kolejnych analizach, gdyż teksty opisowe od respondentów są trudne do analizy.

Wypróbowałam tłumaczenie korzystając z modelu na HuggingFace: https://huggingface.co/Helsinki-NLP/opus-mt-pl-en, jednakże ten model nie jest wytrenowany do dziedziny medycznej, co powoduje, że tłumaczenie nie zawsze było satysfakcjonujące. Wypróbowałam również tłumaczenie przy wykorzystaniu darmowego Ollama3. Wynik również nie był satysfakcjonujący. Najlepszy wynik uzyskałam dzięki tłumaczeniu przez czat gpt 4o-mini. W związku z tym w tym projekcie przedstawiam kod i wynik tłumaczenia z wykorzystaniem gpt 4o-mini.

In [ ]:
api_key = open("C:\\...\\api openai key.txt").read()	# Odczyt klucza api apenai, w miejsce ... ma być lokalizacja
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.0, openai_api_key=api_key)		# Tworzę model, temperatura jest ustawiona na 0.0 dzięki czemu przy kilkukrotnym powtórzeniu kroków, wyniki powinny być powtarzalne.

In [9]:
def translate(description):
	#Tworzę dwa łańcuchy (chains):
    
    # 1. Wykrywanie języka
    template1 = "Return the language this description is written in:\n{description}.\nONLY return the language it was written in."
    prompt1 = ChatPromptTemplate.from_template(template1)
    chain_1 = prompt1|llm
    
    # 2. Tłumaczenie z wykrytego języka na angielski
    template2 = "Translate the following medical description from {language} into English. Ensure that the translation is accurate and uses proper medical terminology. Please provide only the translated text without any additional explanations or statements. Here is the description:\n"+description
    prompt2 = ChatPromptTemplate.from_template(template2)
    chain_2 = prompt2|llm

    language_chain = chain_1
    translation_chain = language_chain|chain_2
    
    return translation_chain.invoke(description)

In [10]:
# Funkcja do tłumaczenia i przetwarzania wyników
def translate_and_parse(description):
    if pd.isna(description):
        return np.nan  # Zwróć NaN, jeśli opis jest NaN
    result = translate(description)
    translate_result_content = result.content
    return translate_result_content

# Zastosowanie funkcji do każdego opisu w DataFrame
data['Tlumaczenie_Ang_Rozpoznanie_pooperacyjne'] = data['Rozpoznanie_pooperacyjne'].apply(translate_and_parse)
data['Tlumaczenie_Ang_Zakres_operacji'] = data['Zakres_operacji'].apply(translate_and_parse)
data['Tlumaczenie_Ang_Rodzaje_dolegliwosci'] = data['Rodzaje_dolegliwosci'].apply(translate_and_parse)
data['Tlumaczenie_Ang_Jakie_inne_metody'] = data['Jakie_inne_metody'].apply(translate_and_parse)
data['Tlumaczenie_Ang_Jakie_inne_choroby'] = data['Jakie_inne_choroby'].apply(translate_and_parse)

In [ ]:
#Sprawdzam jak teraz wygląda dataframe po tłumaczeniu wybranych kolumn
data.head()

In [12]:
#Nowe kolumny są na końcu dataframe. Przestawię je przed odpowiadającymi im kolumnami macierzystymi dla łatwiejszego czytania dataframu.

#Kolumny dataframe:
cols = list(data.columns)

# Przeniesienie kolumny 'Tlumaczenie_Ang_Rozpoznanie_pooperacyjne' przed jej odpowiadającą jej kolumną po polsku
cols.insert(cols.index('Rozpoznanie_pooperacyjne'), cols.pop(cols.index('Tlumaczenie_Ang_Rozpoznanie_pooperacyjne')))

# Przeniesienie kolumny 'Tlumaczenie_Ang_Zakres_operacji' 
cols.insert(cols.index('Zakres_operacji'), cols.pop(cols.index('Tlumaczenie_Ang_Zakres_operacji')))

# Przeniesienie kolumny 'Tlumaczenie_Ang_Rodzaje_dolegliwosci' 
cols.insert(cols.index('Rodzaje_dolegliwosci'), cols.pop(cols.index('Tlumaczenie_Ang_Rodzaje_dolegliwosci')))

# Przeniesienie kolumny 'Tlumaczenie_Ang_Jakie_inne_metody' 
cols.insert(cols.index('Jakie_inne_metody'), cols.pop(cols.index('Tlumaczenie_Ang_Jakie_inne_metody')))

# Przeniesienie kolumny 'Tlumaczenie_Ang_Rodzaje_dolegliwosci' 
cols.insert(cols.index('Jakie_inne_choroby'), cols.pop(cols.index('Tlumaczenie_Ang_Jakie_inne_choroby')))

# Przestawienie DataFrame według zaktualizowanej listy kolumn:
data_transl = data[cols]

In [ ]:
data_transl.head()

Zapis wyniku do pliku. To co będzie obejmowało pracę z opisami pacjentek, będzie wykonywane na przetłumaczonych tekstach na język angielski.

In [14]:
# Zapis do pliku CSV
data_transl.to_csv("C:/Users/justy/Documents/Projekt/01_ankiety_opisy_pacjentek_przetlumaczone.csv", index=False)